In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import argparse
import time
import sys
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/ziniu.wzn/BayesCard")
from DataPrepare.schemas import gen_DB0_schema, gen_stats_light_schema
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
ensemble_location = "/home/ziniu.wzn/stats/BayesCard_temp/"
from DataPrepare.query_prepare_BayesCard import prepare_join_queries
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_CEB_sub_queries.sql"
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path=None, 
                                            query_filename=query_filename, true_card_exist=True)

In [3]:
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path=ensemble_location):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns.append(bn)
            #bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)
with open(query_filename, "rb") as f:
    real_query = f.readlines()
queries = bn_ensemble.parse_query_all(parsed_queries)

2_chow-liu_1.pkl
4_chow-liu_1.pkl
10_chow-liu_1.pkl
8_chow-liu_1.pkl
3_chow-liu_1.pkl
1_chow-liu_1.pkl
6_chow-liu_1.pkl
7_chow-liu_1.pkl
9_chow-liu_1.pkl
5_chow-liu_1.pkl
0_chow-liu_1.pkl


In [4]:
latency = []
q_errors = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q)
    except:
        print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"predicting query no {i}: {real_query[i]} \n")
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

predicting query no 0: b'SELECT COUNT(*) FROM users as u, badges as b WHERE b.UserId= u.Id AND u.UpVotes>=0;||0||79851\n' 

true cardinality 79851, predicted 79850.99999999999 with q-error 1.0000000000000002
predicting query no 1: b"SELECT COUNT(*) FROM badges as b, comments as c, users as u WHERE c.UserId = u.Id AND b.UserId = u.Id AND b.Date<='2014-09-11 14:33:06'::timestamp AND c.Score=0;||1||10220614\n" 

true cardinality 10220614, predicted 11687519.544826282 with q-error 1.143524209487442
predicting query no 2: b'SELECT COUNT(*) FROM postHistory as ph, comments as c, users as u WHERE ph.UserId = u.Id AND c.UserId = u.Id AND ph.PostHistoryTypeId=1 AND c.Score=0;||2||1458075\n' 

true cardinality 1458075, predicted 29194870.56515851 with q-error 20.022886727471843
predicting query no 3: b"SELECT COUNT(*) FROM postHistory as ph, comments as c, users as u WHERE ph.UserId = u.Id AND c.UserId = u.Id AND ph.PostHistoryTypeId=1 AND ph.CreationDate>='2010-09-14 11:59:07'::timestamp;||3||1

predicting query no 24: b'SELECT COUNT(*) FROM postHistory as ph, comments as c, users as u WHERE ph.UserId = u.Id AND c.UserId = u.Id;||11||263105194\n' 

true cardinality 263105194, predicted 258565268.1723978 with q-error 1.017558142513461
predicting query no 25: b"SELECT COUNT(*) FROM users as u, comments as c WHERE c.UserId = u.Id AND u.Reputation>=1 AND u.Reputation<=487 AND u.UpVotes<=27 AND u.CreationDate>='2010-10-22 22:40:35'::timestamp AND u.CreationDate<='2014-09-10 17:01:31'::timestamp;||11||49655\n" 

true cardinality 49655, predicted 56913.114376547004 with q-error 1.146170866509858
predicting query no 26: b"SELECT COUNT(*) FROM users as u, postHistory as ph WHERE ph.UserId = u.Id AND u.Reputation>=1 AND u.Reputation<=487 AND u.UpVotes<=27 AND u.CreationDate>='2010-10-22 22:40:35'::timestamp AND u.CreationDate<='2014-09-10 17:01:31'::timestamp;||11||126748\n" 

true cardinality 126748, predicted 135763.97209013917 with q-error 1.0711330521202636
predicting query no 27: b

predicting query no 57: b"SELECT COUNT(*) FROM posts as p, votes as v WHERE v.PostId = p.Id AND p.Score>=-1 AND p.CreationDate>='2010-10-21 13:21:24'::timestamp AND p.CreationDate<='2014-09-09 15:12:22'::timestamp AND v.CreationDate<='2014-09-12 00:00:00'::timestamp;||20||288438\n" 

true cardinality 288438, predicted 288684.2940618952 with q-error 1.00085388909192
predicting query no 58: b"SELECT COUNT(*) FROM users as u, votes as v WHERE v.UserId = u.Id AND u.UpVotes>=0 AND u.CreationDate>='2010-07-27 17:15:57'::timestamp AND u.CreationDate<='2014-09-03 12:47:42'::timestamp AND v.CreationDate<='2014-09-12 00:00:00'::timestamp;||20||31965\n" 

true cardinality 31965, predicted 31927.878621375196 with q-error 1.0011626634849442
predicting query no 59: b"SELECT COUNT(*) FROM users as u, votes as v, posts as p WHERE v.UserId = u.Id AND v.PostId = p.Id AND u.UpVotes>=0 AND u.CreationDate>='2010-07-27 17:15:57'::timestamp AND u.CreationDate<='2014-09-03 12:47:42'::timestamp AND v.CreationD

predicting query no 90: b"SELECT COUNT(*) FROM users as u, comments as c, posts as p, postLinks as pl WHERE p.Id = pl.RelatedPostId AND p.OwnerUserId = u.Id AND p.Id = c.PostId AND u.UpVotes>=0 AND u.CreationDate>='2010-08-21 21:27:38'::timestamp AND c.CreationDate>='2010-07-21 11:05:37'::timestamp AND c.CreationDate<='2014-08-25 17:59:25'::timestamp;||25||25033\n" 

true cardinality 25033, predicted 24519.877662688294 with q-error 1.0209267902707573
predicting query no 91: b"SELECT COUNT(*) FROM posts as p, comments as c, users as u WHERE u.Id = p.OwnerUserId AND u.Id = c.UserId AND p.CreationDate>='2010-07-27 01:51:15'::timestamp AND c.Score=0 AND c.CreationDate<='2014-09-13 20:12:15'::timestamp;||26||36980622\n" 

true cardinality 36980622, predicted 36069598.73957365 with q-error 1.0252573716443045
predicting query no 92: b"SELECT COUNT(*) FROM votes as v, comments as c, users as u WHERE u.Id = c.UserId AND u.Id = v.UserId AND v.BountyAmount<=50 AND v.CreationDate<='2014-09-12 00:0

predicting query no 123: b'SELECT COUNT(*) FROM users as u, badges as b WHERE u.Id = b.UserId;||29||79851\n' 

true cardinality 79851, predicted 79851.0 with q-error 1.0
predicting query no 124: b"SELECT COUNT(*) FROM badges as b, votes as v, posts as p, users as u WHERE u.Id = p.OwnerUserId AND u.Id = v.UserId AND u.Id = b.UserId AND p.Score>=0 AND p.Score<=30 AND p.CommentCount=0 AND p.CreationDate>='2010-07-27 15:30:31'::timestamp AND p.CreationDate<='2014-09-04 17:45:10'::timestamp;||29||232039659\n" 

true cardinality 232039659, predicted 228117695.5325055 with q-error 1.0171927191283396
predicting query no 125: b"SELECT COUNT(*) FROM users as u, votes as v, posts as p WHERE u.Id = p.OwnerUserId AND u.Id = v.UserId AND p.Score>=0 AND p.Score<=30 AND p.CommentCount=0 AND p.CreationDate>='2010-07-27 15:30:31'::timestamp AND p.CreationDate<='2014-09-04 17:45:10'::timestamp;||29||1187068\n" 

true cardinality 1187068, predicted 1131929.0357604816 with q-error 1.0487123861103833
predic

...

predicting query no 2513: b"SELECT COUNT(*) FROM badges as b, comments as c, posts as p, postHistory as ph, votes as v, users as u WHERE u.Id = b.UserId AND p.Id = v.PostId AND p.Id = ph.PostId AND u.Id = p.OwnerUserId AND p.Id = c.PostId AND b.Date<='2014-09-11 21:46:02'::timestamp AND p.AnswerCount>=0 AND p.CommentCount>=0;||143||71123272\n" 

true cardinality 71123272, predicted 47024932.323016986 with q-error 1.5124587848728865
predicting query no 2514: b'SELECT COUNT(*) FROM users as u, comments as c, posts as p, postHistory as ph, votes as v WHERE p.Id = ph.PostId AND p.Id = c.PostId AND u.Id = p.OwnerUserId AND p.Id = v.PostId AND u.Reputation>=1 AND u.Reputation<=642 AND u.DownVotes>=0 AND p.AnswerCount>=0 AND p.CommentCount>=0;||143||2087179\n' 

true cardinality 2087179, predicted 917711.5209662879 with q-error 2.2743301705554946
predicting query no 2515: b"SELECT COUNT(*) FROM users as u, comments as c, posts as p, postHistory as ph, badges as b WHERE p.Id = ph.PostId AND u.

predicting query no 2541: b"SELECT COUNT(*) FROM users as u, posts as p, postHistory as ph WHERE p.Id = ph.PostId AND u.Id = p.OwnerUserId AND u.UpVotes>=0 AND u.CreationDate<='2014-09-11 20:31:48'::timestamp AND p.PostTypeId=1 AND p.AnswerCount>=0 AND p.CreationDate>='2010-07-21 15:23:53'::timestamp AND p.CreationDate<='2014-09-11 23:26:14'::timestamp AND ph.PostHistoryTypeId=5;||144||38044\n" 

true cardinality 38044, predicted 51546.02979123406 with q-error 1.3549056300923683
predicting query no 2542: b"SELECT COUNT(*) FROM users as u, posts as p, votes as v WHERE u.Id = p.OwnerUserId AND p.Id = v.PostId AND u.UpVotes>=0 AND u.CreationDate<='2014-09-11 20:31:48'::timestamp AND p.PostTypeId=1 AND p.AnswerCount>=0 AND p.CreationDate>='2010-07-21 15:23:53'::timestamp AND p.CreationDate<='2014-09-11 23:26:14'::timestamp AND v.CreationDate>='2010-07-21 00:00:00'::timestamp;||144||146996\n" 

true cardinality 146996, predicted 92658.3980591461 with q-error 1.5864293262027787
predicting qu

predicting query no 2556: b"SELECT COUNT(*) FROM users as u, comments as c, posts as p, postLinks as pl, postHistory as ph WHERE p.Id = pl.PostId AND p.Id = ph.PostId AND p.Id = c.PostId AND u.Id = p.OwnerUserId AND u.UpVotes>=0 AND u.CreationDate<='2014-09-11 20:31:48'::timestamp AND p.PostTypeId=1 AND p.AnswerCount>=0 AND p.CreationDate>='2010-07-21 15:23:53'::timestamp AND p.CreationDate<='2014-09-11 23:26:14'::timestamp AND pl.CreationDate>='2010-11-16 01:27:37'::timestamp AND pl.CreationDate<='2014-08-21 15:25:23'::timestamp AND ph.PostHistoryTypeId=5;||144||50824\n" 

true cardinality 50824, predicted 14609.672729755614 with q-error 3.478791136538359
predicting query no 2557: b"SELECT COUNT(*) FROM users as u, comments as c, posts as p, postLinks as pl, votes as v WHERE p.Id = pl.PostId AND p.Id = c.PostId AND u.Id = p.OwnerUserId AND p.Id = v.PostId AND u.UpVotes>=0 AND u.CreationDate<='2014-09-11 20:31:48'::timestamp AND p.PostTypeId=1 AND p.AnswerCount>=0 AND p.CreationDate>='

predicting query no 2575: b"SELECT COUNT(*) FROM users as u, comments as c, posts as p WHERE p.Id = c.PostId AND u.Id = p.OwnerUserId AND u.DownVotes>=0 AND u.UpVotes>=0 AND c.CreationDate>='2010-07-26 19:37:03'::timestamp AND p.Score>=-2 AND p.CommentCount<=18 AND p.CreationDate>='2010-07-21 13:50:08'::timestamp AND p.CreationDate<='2014-09-11 00:53:10'::timestamp;||145||167410\n" 

true cardinality 167410, predicted 167697.37964941494 with q-error 1.0017166217634248
predicting query no 2576: b"SELECT COUNT(*) FROM postHistory as ph, comments as c, postLinks as pl, posts as p WHERE p.Id = pl.PostId AND p.Id = ph.PostId AND p.Id = c.PostId AND ph.CreationDate>='2010-11-27 03:38:45'::timestamp AND c.CreationDate>='2010-07-26 19:37:03'::timestamp AND pl.CreationDate<='2014-08-05 18:27:51'::timestamp;||145||204137\n" 

true cardinality 204137, predicted 138226.8543049814 with q-error 1.476825910756788
predicting query no 2577: b"SELECT COUNT(*) FROM votes as v, comments as c, postLinks as

predicting query no 2597: b"SELECT COUNT(*) FROM votes as v, comments as c, posts as p, postLinks as pl, postHistory as ph WHERE p.Id = pl.PostId AND p.Id = ph.PostId AND p.Id = c.PostId AND p.Id = v.PostId AND c.CreationDate>='2010-07-26 19:37:03'::timestamp AND p.Score>=-2 AND p.CommentCount<=18 AND p.CreationDate>='2010-07-21 13:50:08'::timestamp AND p.CreationDate<='2014-09-11 00:53:10'::timestamp AND pl.CreationDate<='2014-08-05 18:27:51'::timestamp AND ph.CreationDate>='2010-11-27 03:38:45'::timestamp;||145||1586326\n" 

true cardinality 1586326, predicted 1020351.178308415 with q-error 1.5546863018572528
predicting query no 2598: b"SELECT COUNT(*) FROM users as u, comments as c, posts as p, postLinks as pl, postHistory as ph WHERE p.Id = pl.PostId AND p.Id = ph.PostId AND p.Id = c.PostId AND u.Id = p.OwnerUserId AND u.DownVotes>=0 AND u.UpVotes>=0 AND c.CreationDate>='2010-07-26 19:37:03'::timestamp AND p.Score>=-2 AND p.CommentCount<=18 AND p.CreationDate>='2010-07-21 13:50:08'

In [ ]:
sys.path.append('/home/ziniu.wzn/FSPN/fspn')
from Data_prepare.schemas import gen_stats_light_schema
from Data_prepare.query_prepare_BayesCard import prepare_join_queries
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
hdf_path = "/home/ziniu.wzn/stats/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'

ensemble_location = "/home/ziniu.wzn/stats/FSPN_ensemble/"
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_sub_queries.sql"
#query_filename = "/home/ziniu.wzn/deepdb-public/benchmarks/job-light/sql/job_light_queries.sql"
pairwise_rdc_path = "/home/yuxing.hyx/repository/imdb-benchmark/spn_ensembles/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=True, true_card_exist=True)

In [ ]:
sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path="/home/ziniu.wzn/stats/BayesCard_ensemble/"):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)

In [ ]:
with open(query_filename, "rb") as f:
    real_query = f.readlines()

queries = bn_ensemble.parse_query_all(parsed_queries)


In [ ]:
latency = []
q_errors = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q)
    except:
        print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"predicting query no {i}: {real_query[i]} \n")
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

In [ ]:
parsed_queries[1869]

In [ ]:
q_errors = np.asarray(q_errors)

In [ ]:
np.where(q_errors > 5000)

In [ ]:
q = queries[1869]

In [ ]:
print(len(q))
q

In [ ]:
real_query[1869]

In [ ]:
bn_ensemble.bns[1].table_name

In [ ]:
bn_ensemble.bns[0].

In [ ]:
parsed_queries[1869]

In [ ]:
p, _ = bn_ensemble.bns[1].query(q[1]["query"], n_distinct=q[1]["n_distinct"], return_prob=True)

In [ ]:
p

In [ ]:
p, _ = bn_ensemble.bns[1].expectation(q[1]["query"], e, n_distinct=q[1]["n_distinct"], return_prob=True)

In [ ]:
q[0]*p

In [ ]:
e = ['posts.mul_postHistory.PostId', 'posts.mul_comments.PostId_nn']

In [ ]:
bn_ensemble.bns[1].fanout_attr_inverse

In [ ]:
pred = np.load("/home/ziniu.wzn/stats/est_results/bayesCard_join.npy")

In [ ]:
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_CEB_sub_queries.sql"
with open(query_filename, "r") as f:
    queries = f.readlines()
true = [int(q.split("||")[-1]) for q in queries]
true = np.asarray(true)

In [ ]:
q_errors = np.maximum(true/pred, pred/true)

In [ ]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")

In [ ]:
pred2 = pred + np.random.normal(10, 100, size=2603)
pred2[pred2<=1] = 1

In [ ]:
q_errors = np.maximum(true/pred2, pred2/true)
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")

In [ ]:
np.save("/home/ziniu.wzn/stats/est_results/bayesCard_join_update", pred2)